In [12]:
from scipy.integrate import odeint
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 

#!pip install mpld3
import mpld3
mpld3.enable_notebook()

In [2]:
def plotseird(t, S, E, I, R, D=None, L=None, R0=None, Alpha=None, CFR=None):
    f, ax = plt.subplots(1,1,figsize=(10,4))
    ax.plot(t, S, 'b', alpha=0.7, linewidth=2, label='Susceptible')
    #ax.plot(t, E, 'y', alpha=0.7, linewidth=2, label='Exposed')
    ax.plot(t, I, 'r', alpha=0.7, linewidth=2, label='Infected')
    ax.plot(t, R, 'g', alpha=0.7, linewidth=2, label='Recovered')
    if D is not None:
        ax.plot(t, D, 'k', alpha=0.7, linewidth=2, label='Dead')
        ax.plot(t, S+E+I+R+D, 'c--', alpha=0.7, linewidth=2, label='Total')
    else:
        ax.plot(t, S+E+I+R, 'c--', alpha=0.7, linewidth=2, label='Total')

    ax.set_xlabel('Time (days)')

    ax.yaxis.set_tick_params(length=0)
    ax.xaxis.set_tick_params(length=0)
    ax.grid(b=True, which='major', c='w', lw=2, ls='-')
    legend = ax.legend(borderpad=2.0)
    legend.get_frame().set_alpha(0.5)
    for spine in ('top', 'right', 'bottom', 'left'):
        ax.spines[spine].set_visible(False)
    if L is not None:
        plt.title("Lockdown after {} days".format(L))
    plt.show();

    if R0 is not None or CFR is not None:
        f = plt.figure(figsize=(12,4))

    if R0 is not None:
        # sp1
        ax1 = f.add_subplot(121)
        ax1.plot(t, R0, 'b--', alpha=0.7, linewidth=2, label='R_0')

        ax1.set_xlabel('Time (days)')
        ax1.title.set_text('R_0 over time')
        # ax.set_ylabel('Number (1000s)')
        # ax.set_ylim(0,1.2)
        ax1.yaxis.set_tick_params(length=0)
        ax1.xaxis.set_tick_params(length=0)
        ax1.grid(b=True, which='major', c='w', lw=2, ls='-')
        legend = ax1.legend()
        legend.get_frame().set_alpha(0.5)
        for spine in ('top', 'right', 'bottom', 'left'):
            ax.spines[spine].set_visible(False)

    if Alpha is not None:
        # sp2
        ax2 = f.add_subplot(122)
        ax2.plot(t, Alpha, 'r--', alpha=0.7, linewidth=2, label='alpha')

        ax2.set_xlabel('Time (days)')
        ax2.title.set_text('fatality rate over time')
        # ax.set_ylabel('Number (1000s)')
        # ax.set_ylim(0,1.2)
        ax2.yaxis.set_tick_params(length=0)
        ax2.xaxis.set_tick_params(length=0)
        ax2.grid(b=True, which='major', c='w', lw=2, ls='-')
        legend = ax2.legend()
        legend.get_frame().set_alpha(0.5)
        for spine in ('top', 'right', 'bottom', 'left'):
            ax.spines[spine].set_visible(False)

    plt.show();

In [3]:
# Create SEIR PDEs

def deriv(y, t, N, beta, gamma, delta, alpha, rho):
    S, E, I, R, D = y
    dSdt = -beta * S * I / N
    dEdt = beta * S * I / N - delta * E
    dIdt = delta * E - (1 - alpha) * gamma * I - alpha * rho * I
    dRdt = (1 - alpha) * gamma * I
    dDdt = alpha * rho * I
    return dSdt, dEdt, dIdt, dRdt, dDdt

In [4]:
# Load infections, deaths and recover data into dataframes

import pandas as pd

africa_infections_url = "https://raw.githubusercontent.com/dsfsi/covid19africa/master/data/time_series/africa_daily_time_series_cases.csv"
africa_deaths_url = "https://raw.githubusercontent.com/dsfsi/covid19africa/master/data/time_series/africa_daily_time_series_deaths.csv"
africa_recoveries_url = "https://raw.githubusercontent.com/dsfsi/covid19africa/master/data/time_series/africa_daily_time_series_recovered.csv"

africa_infections_df = pd.read_csv(africa_infections_url).transpose()
africa_infections_df.columns = africa_infections_df.iloc[0]
africa_infections_df = africa_infections_df.drop("Country/Region")
africa_infections_df = africa_infections_df.drop(["Lat","Long"])

africa_deaths_df = pd.read_csv(africa_deaths_url).transpose()
africa_deaths_df.columns = africa_deaths_df.iloc[0]
africa_deaths_df = africa_deaths_df.drop("Country/Region")
africa_deaths_df = africa_deaths_df.drop(["Lat","Long"])

africa_recoveries_df = pd.read_csv(africa_recoveries_url).transpose()
africa_recoveries_df.columns = africa_recoveries_df.iloc[0]
africa_recoveries_df = africa_recoveries_df.drop("Country/Region")
africa_recoveries_df = africa_recoveries_df.drop(["Lat","Long"])


In [5]:
# Change date format

africa_infections_df.index = pd.to_datetime(africa_infections_df.index)
africa_deaths_df.index = pd.to_datetime(africa_deaths_df.index)
africa_recoveries_df.index = pd.to_datetime(africa_recoveries_df.index)

africa_recoveries_df.head()

Country/Region,Algeria,Angola,Benin,Burkina Faso,Cabo Verde,Cameroon,CAR,Chad,Congo,DRC,...,Uganda,Zambia,Zimbabwe,Botswana,Burundi,Sierra Leone,Malawi,South Sudan,Western Sahara,Sao Tome and Principe
2020-01-22,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-01-23,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-01-24,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-01-25,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-01-26,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
# Load population data

pop_url = "https://raw.githubusercontent.com/datasets/population/master/data/population.csv"
pop_df = pd.read_csv(pop_url)

In [7]:
# install widgets

#!pip install ipywidgets
#!jupyter nbextension enable --py widgetsnbextension

In [11]:

import ipywidgets as widgets
from ipywidgets import interact, interact_manual
import datetime

@interact 
def plot_SEIR(country = africa_infections_df.columns,
              date = widgets.DatePicker(value=pd.to_datetime(africa_infections_df.index[-1])),
              D = widgets.FloatText(value= 14), # how many days infection lasts for
              incubation_period =  widgets.FloatText(value= 1), # number of days an exposed individual becomes infectious
              alpha = widgets.FloatText(value=0.002), # death rate
              no_days_to_death = widgets.FloatText(value=14)          
             ):
    
                date = date.strftime("%Y-%m-%d")
                infections = africa_infections_df.loc[date, country]
                no_deaths = africa_deaths_df.loc[date, country]
                no_recoveries = africa_recoveries_df.loc[date, country]
                R_0 = infections/no_recoveries

                N = pop_df.loc[(pop_df["Country Name"] == country) & (pop_df["Year"] == 2018)]["Value"].values
                
                gamma = 1.0 / D
                delta = 1.0 / incubation_period # incubation period of five days
                beta = R_0 * gamma  # R_0 = beta / gamma, so beta = R_0 * gamma
                rho = 1/no_days_to_death  # days from infection until death    
                S0, E0, I0, R0, D0 = N-15515, 264, 15515, 7006, 264  # initial conditions: one exposed
                
                t = np.linspace(0, 600, 150) # Grid of time points (in days)
                y0 = S0, E0, I0, R0, D0 # Initial conditions v
    
                ret = odeint(deriv, y0, t, args=(N, beta, gamma, delta, alpha, rho))
                S, E, I, R, D = ret.T
                plotseird(t, S, E, I, R, D)
            
                return

interactive(children=(Dropdown(description='country', options=('Algeria', 'Angola', 'Benin', 'Burkina Faso', '…